## 2. Datenvorverarbeitung mit Pandas: Food Nutrient Database

Diese Aufgabe befasst sich mit einer deutlich umfangreicheren Datenbank des *U.S. Department of Agriculture*, aufbereitet im Format JSON von A. Williams, zum Thema Nährstoffgehalt von Nahrungsmitteln. Sie enthält 6636 Einträge für Nahrungsmittel, alle in Form von JSON-Records, wie z.B.:

```
{
    "id": 21441,
    "description": "KENTUCKY FRIED CHICKEN, Fried Chicken, EXTRA CRISPY, Wing,
        meat and skin with breading", "tags": ["KFC"],
    "manufacturer": "Kentucky Fried Chicken", "group": "Fast Foods",
    "portions": [ 
        {
            "amount": 1,
            "unit": "wing , with skin",
            "grams": 68.0 
        },
        ...
    ], 
    "nutrients": [
        {
            "value": 20.8,
            "units": "g", 
            "description": "Protein",
            "group": "Composition" 
        },
        ... 
    ]
}
```

Ziel der Analyse in dieser Übung ist es, eine explorative Analyse des Gehalts des Spurenelementes Zink in den verschiedenen Nahrungsmitteln durchzuführen. Notwendig dafür sind etwas aufwändigere, aber für die Datenanlyse typische Manipulationen mit Pandas sowie der
Einsatz zusätzlicher Python-Standardbibliotheken zum Download und der Verarbeitung von Zip- und JSON-Dateien.

Aufgaben:


### 2.1 Laden und Einlesen der Datenbank

a) Laden Sie die Datenbank als zip-File aus Moodle herunter und lesen Sie dieses File direkt in ein neues Notebook ein. Die bisher verwendete Pandas-Methode `read_csv()` funktioniert für JSON-Files leider nicht. Das heruntergeladene File wird stattdessen mithilfe des Pythonmoduls `zipfile` entpackt und dem Python-Befehl `open()` eingelesen. Die Umwandlung des JSON-Formates in ein geeignetes Python-Format erfolgt mit einem weiteren Modul der Python-Standardlibrary, `json`, hier mithilfe der Funktion `json.load()`. Lesen Sie dazu die zugehörigen, auf dem Web bzw. Stackoverflow verfügbaren Anleitungen.

#### 2.1.1 Einlesen und Entpacken des Zip-Files

Wie im Folgenden gezeigt, wird das Zip-File entpackt und die JSON-Daten eingelesen. Hierbei wurde sich nach einigem herumprobieren dafür entschieden, die in der JSON Datei vorhanden Dictionaries in einem übergeordneten Dictionary zu speichern, wobei die bereits vorhanden IDs der JSON Records als Keys verwendet werden.

In [ ]:
import zipfile
import json

# path to the zip file
zip_file_path = 'foods-2011-10-03.json.zip'

records = {}

with zipfile.ZipFile(zip_file_path, 'r') as z:
	file_list = [n for n in z.namelist() if not n.endswith('/')]  # nur Dateien, keine Verzeichnisse
	for file_name in file_list:
		with z.open(file_name) as f:
			data = json.load(f)
			if isinstance(data, list):
				for record in data:
					records[record['id']] = record

print(f"Anzahl der eingelesenen JSON-Records: {len(records)}")

prüfen ob das so geklappt hat wie gewünscht 

In [ ]:
print(f"type of records: {type(records)}")

Anzeigen aller in der Datenbank enthaltenen Keys

In [ ]:
records.keys()

Und dann mal noch schauen ob der aus der Aufgabenbeschreibung genannte Eintrag mit der ID 21441 auch wirklich Fried Chicken ist :)

In [ ]:
records.get(21441)

___

### 2.2 Aufbereitung der Datenbank

b) Die Datenbank steht nun in Form einer Liste aus 6636 Python-Dictionaries zu Verfügung. Jedes Dictionary enthält Angaben zu einem Nahrungsmittel. Greifen Sie sich ein beliebiges Nahrungsmittel heraus und lassen sich die Namen der Einträge mit der Methode `dict.keys()` anzeigen. Einer der Einträge enthält die enthaltenen Nährstoffe (`nutrients`), ebenfalls als Dictionary. Lassen Sie sich wiederum einen beliebigen Eintrag der Nährstoffliste anzeigen. Es sollte auffallen, dass manche Feldnamen doppelt vorkommen.

Teile dieser hierarchischen Struktur sollen nun in eine einheitliche Tabelle umgewandelt werden, um eine explorative Analyse durchführen zu können.

Vorgehensweise:

* Kopieren Sie zunächst die Felder `description`,`group`,`id`,`manufacturer` in einen eigenen DataFrame `info`, sowie alle Nährstofflisten in ein Array von DataFrames, wobei Sie an jeden DataFrame die entsprechende ID des Nahrungsmittels als eigene Spalte anhängen.
* Dieses Array wird mithilfe der Funktion `pandas.concat()` zu einem großen DataFrame nutrients (389355 Einträge) vereinigt.
* Entfernen Sie alle Duplikate aus diesem DataFrame.
* Bevor beide DataFrames vereinigt werden können, gibt es noch ein Problem: beide enthalten Felder mit dem Namen `description` und `group` (s.o.). Benennen Sie diese daher mithilfe von DataFrame.rename() in eindeutige Namen um.
* Vereinigen Sie beide DataFrames mit `pandas.merge(nutrients, info, on=’id’, how=’outer’)` anhand der Nahrungsmittel-ID.

Überprüfen Sie das Ergebnis jeder Manipulation mit `DataFrame.head()``.

#### 2.2.1 Anzeigen eines beliebigen Eintrags der Nährstoffliste

In [ ]:
records.get(11444).keys()

In [ ]:
import pprint

#record = records.get(11444)
record = records.get(21441)

pprint.pprint(record.get('nutrients'), width=120, sort_dicts=True)

#### 2.2.2 Umwandeln in DataFrames

##### DataFrames erstellen

Kopieren Sie zunächst die Felder `description`,`group`,`id`,`manufacturer` in einen eigenen DataFrame `info`...

In [ ]:
import pandas as pd

info = pd.DataFrame([
    {
        'id': r.get('id'),
        'description': r.get('description'),
        'group': r.get('group'),
        'manufacturer': r.get('manufacturer')
    }
    for r in records.values()
])

info.head()

... sowie alle Nährstofflisten in ein Array von DataFrames, wobei Sie an jeden DataFrame die entsprechende ID des Nahrungsmittels als eigene Spalte anhängen.

In [ ]:
nutrient_dfs = []
for r in records.values():
    nd = pd.DataFrame(r.get('nutrients', []))
    if not nd.empty:
        nd['id'] = r['id']
        nutrient_dfs.append(nd)
        
nutrient_dfs[0].head()

Dieses Array wird mithilfe der Funktion `pandas.concat()` zu einem großen DataFrame nutrients (389355 Einträge) vereinigt

In [ ]:
nutrients = pd.concat(nutrient_dfs, ignore_index=True)
print(nutrients.shape) # show the shape of the combined DataFrame (should be (389355, 5))

Entfernen Sie alle Duplikate aus diesem DataFrame.

In [ ]:
nutrients.drop_duplicates(inplace=True) # inplace=True to modify the DataFrame directly
print(nutrients.shape) # show the shape after removing duplicates

Bevor beide DataFrames vereinigt werden können, gibt es noch ein Problem: beide enthalten Felder mit dem Namen `description` und `group` (s.o.). Benennen Sie diese daher mithilfe von DataFrame.rename() in eindeutige Namen um. Dies soll nochmal anhand der beiden DataFrames `info` und `nutrients` gezeigt werden:

In [ ]:
nutrients.rename(columns={'description': 'nutrient_description', 'group': 'nutrient_group'}, inplace=True)
info.rename(columns={'description': 'food_description', 'group': 'food_group'}, inplace=True)
nutrients.head()

In [ ]:
info.head()

Vereinigen Sie beide DataFrames mit `pandas.merge(nutrients, info, on=’id’, how=’outer’)` anhand der Nahrungsmittel-ID.

In [ ]:
df = pd.merge(nutrients, info, on='id', how='outer') # how='outer' to keep all records
df.head()

In [ ]:
df.shape

___

### 2.3 Untersuchung des Spurenelements Zink

c) Nun sind die Daten bereit für die Untersuchung auf das Spurenelement Zink (Feldname: `Zinc, Zn`). Lesen Sie dazu alle Tabelleneinträge mithilfe einer geeigneten Indizierung in einen DataFrame aus, der nur Einträge zum Nährstoff Zink enthält...

#### 2.3.1 Auslesen der Zink-Einträge

In [ ]:
zn = pd.DataFrame(df[df['nutrient_description'] == 'Zinc, Zn'])
zn.head()

In [ ]:
zn.shape

Eine Untersuchung der Daten hat ergeben, dass Einträge zum Nährstoff Zink in `mg` vorliegen weshalb keine weitere Verarbeitung erforderlich ist.

#### 2.3.2 Explorative Statistiken des Zinkgehalts

... Daraus wählen Sie wiederum die Spalte mit dem Zinkgehalt in mg (`value`) aus und stellen dafür ein Histogramm und eine Liste deskriptiver Statistiken dar.

##### 2.3.2.1 Histogramm des Zinkgehalts in mg

In [ ]:
import matplotlib.pyplot as plt

# histogram of zinc content in mg
plt.figure(figsize=(10,6))
plt.hist(zn['value'].dropna(), bins=50, color='skyblue', edgecolor='black')
plt.title('Histogram of Zinc Content (mg)')
plt.xlabel('Zinc Content (mg)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

##### 2.3.2.2 Deskriptive Statistiken des Zinkgehalts in mg

In [ ]:
import numpy as np
from scipy import stats

# basic stuff like mean, median, std, min, max, percentiles
desc = zn['value'].describe()
print("\nDeskriptive Statistiken:")
print(desc)

In [ ]:
# get the variance (std^2), skewness, and kurtosis
variance = np.var(zn['value'].dropna())
skewness = stats.skew(zn['value'].dropna())
kurtosis = stats.kurtosis(zn['value'].dropna())
print(f"\nVarianz: {variance:.4f}")
print(f"Schiefe: {skewness:.4f}")
print(f"Kurtosis: {kurtosis:.4f}")

##### 2.3.2.3 Visualisierung der Ergebnisse

In [ ]:
plt.figure(figsize=(10,6))
plt.boxplot(zn['value'].dropna(), vert=False, showmeans=True, patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='black'),
            medianprops=dict(color='orange'),
            meanprops=dict(marker='D', markeredgecolor='red', markerfacecolor='red'))
plt.xlabel('Zink (mg)')
plt.title('Boxplot des Zinkgehalts (mg)')

plt.show()


In [ ]:
import seaborn as sns
import scipy.stats as st

vals = zn['value'].dropna()
n = len(vals)
variance = vals.var(ddof=1)
std = vals.std(ddof=1)
skewness = st.skew(vals)
kurtosis = st.kurtosis(vals)  # Fisher: 0 für Normalverteilung

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# 1) Histogramm + KDE (oben links)
ax = axs[0, 0]
sns.histplot(vals, bins=50, kde=True, color='skyblue', ax=ax)
ax.axvline(vals.mean(), color='red', linestyle='--', label=f'Mean {vals.mean():.2f}')
ax.axvline(vals.median(), color='orange', linestyle='-', label=f'Median {vals.median():.2f}')
ax.set_title('Histogramm + KDE')
ax.set_xlabel('Zink (mg)')
ax.legend()

# 2) Violinplot + inner box (oben rechts)
ax = axs[0, 1]
sns.violinplot(x=vals, inner='quartile', color='lightgreen', ax=ax)
ax.set_title('Violinplot (Verteilung, Median, Quartile)')
ax.set_xlabel('Zink (mg)')

# 3) Q-Q-Plot gegen Normalverteilung (unten links)
ax = axs[1, 0]
st.probplot(vals, dist='norm', plot=ax)
ax.set_title('Q‑Q‑Plot vs Normalverteilung')

# 4) Textbox mit Kennzahlen und kurzer Interpretation (unten rechts)
ax = axs[1, 1]
ax.axis('off')
text = (
    f"n = {n}\n"
    f"Mean = {vals.mean():.3f} mg\n"
    f"Median = {vals.median():.3f} mg\n"
    f"Varianz (sample) = {variance:.3f}\n"
    f"Std (sample) = {std:.3f}\n"
    f"Skewness = {skewness:.3f}\n"
    f"Kurtosis (Fisher) = {kurtosis:.3f}\n\n"
    "Interpretation:\n"
    "- Skewness > 0 → rechtssteil / langer rechter Schwanz\n"
    "- Kurtosis > 0 → stärkere Ausreißer / schwerere Tails als Normal\n"
)
ax.text(0.01, 0.98, text, va='top', ha='left', fontsize=11, family='monospace')

plt.suptitle('Zink: Verteilung, Varianz, Schiefe und Kurtosis', fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#### 2.3.3 Die Edamer-Frage

... Finden Sie in Ihrer Tabelle Edamer (`Cheese, edam`). Hat Edamer einen überdurchschnittlichen Zinkgehalt?

In [ ]:
# find data frame entries where food_description contains 'Cheese, edam'
cheese_edam = zn[zn['food_description'].str.contains('Cheese, edam', case=False, na=False)]
cheese_edam.head()

In [ ]:
edam_mean = cheese_edam['value'].mean() # not necessary to get mean cause only one edam entry
zn_mean = zn['value'].mean()

print(f"Durchschnittlicher Zinkgehalt von Edamer: {edam_mean:.2f} mg\nDurchschnitt aller Nahrungsmittel: {zn_mean:.2f} mg")

Haben mehr als 75% aller Nahrungsmittel einen kleineren Zinkgehalt?

In [ ]:
smaller = (zn['value'] < edam_mean).sum()
print(f"Anzahl der Nahrungsmittel mit kleinerem Zinkgehalt als Edamer: {smaller}")
print(f"Haben mehr als 75% aller Nahrungsmittel einen kleineren Zinkgehalt als Edamer? {'Ja' if smaller > 0.75 * len(zn) else 'Nein'}")



In [ ]:
plt.figure(figsize=(10,6))
plt.hist(zn['value'].dropna(), bins=100, color='skyblue', edgecolor='black')
plt.title('Histogram of Zinc Content (mg)')
plt.xlabel('Zinc Content (mg)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.axvline(edam_mean, color='red', linestyle='--', label=f'Edamer Mean {edam_mean:.2f} mg')
plt.legend()
plt.show()

Welches Nahrungsmittel hat den maximalen Zinkgehalt?

In [ ]:
zn_max = zn.get(zn['value'] == zn['value'].max())
zn_max

Alternativ lässt sich ein Eintrag in einem Pandas Data-Frame mittels der Methode `loc[]` finden, indem man den Zeilenindex angibt, der mit `idxmax()` für die Spalte `value` des DataFrames `zn` ermittelt werden kann.

In [ ]:
max_row = zn.loc[zn['value'].idxmax()]
max_row

und noch eine entscheidende Frage die hier noch fehlt, hat Edamer mehr Zink als andere Käsesorten im Durchschnitt?

In [1]:
cheese = zn[zn['food_description'].str.contains('Cheese', case=False, na=False)]
cheese.head()

cheese_mean = cheese['value'].mean()
print(f"Durchschnittlicher Zinkgehalt von Käse: {cheese_mean:.2f} mg\nDurchschnitt Edamer: {edam_mean:.2f} mg\nDurchschnitt aller Nahrungsmittel: {zn_mean:.2f} mg")

NameError: name 'zn' is not defined